In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import subprocess
import shlex
import numpy as np
import pandas as pd

import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(repo_root)

from sklearn.model_selection import train_test_split
from src.utils.file_handlers import group_dataset_files, read_dataset_files
from src.utils.operation_helpers import run_models, split_games
from src.utils.c_operation_helpers import  run_simulation
from src.utils.file_handlers import process_directory_real_data
from src.models.BradleyTerry import compute_predicted_ratings_HOL_BT

In [ ]:
EXPERIMENT_PATH = os.path.join(repo_root, 'exp', 'ex03')
os.chdir(EXPERIMENT_PATH)

### Example

In [2]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

edge_file = grouped['00009']['edges']
node_file = grouped['00009']['nodes']
edge_path = os.path.join(data_dir, edge_file)
node_path = os.path.join(data_dir, node_file)
print(edge_path)
print(node_path)

results = run_simulation(node_path, edge_path,ratio=.8, is_synthetic=0)
print(results)

/home/jackyeung99/senior_thesis/higher_order_ranking/datasets/Real_Data/00009_edges.txt
/home/jackyeung99/senior_thesis/higher_order_ranking/datasets/Real_Data/00009_nodes.txt
    model  av_error         rho         tau     prior log-likelihood  \
0   HO_BT  0.577566  -0.0357143  0.00952381  -4.16649     -0.0547989   
1  HOL_BT  0.577566  -0.0357143  0.00952381  -4.16649     -0.0547989   
2     BIN  0.577566  -0.0357143  0.00952381  -4.16649     -0.0547989   
3    BINL  0.577566  -0.0357143  0.00952381  -4.16649     -0.0547989   

  leadership-log-likelihood iterations  
0                -0.0547989         21  
1                -0.0547989         21  
2                -0.0547989         21  
3                -0.0547989         21  


### Full Results

In [5]:
process_directory_real_data(EXPERIMENT_PATH)

In [6]:


def process_results():
    # Read the summary CSV files
    log_likelihood_df = pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()
    leadership_log_likelihood_df = pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'leadership_log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()

    filtered_log_like = log_likelihood_df.drop(columns=['epoch'])
    filtered_leader_like = leadership_log_likelihood_df.drop(columns=['epoch'])
   
    log_like = subtract_columns(filtered_log_like, 'HO_BT')
    leadership_log = subtract_columns(filtered_leader_like, 'HOL_BT')


    return log_like, leadership_log

def subtract_columns(df, compared_col):
    columns = df.columns.tolist()
    # base_column = columns[[compared_col]] 
    for col in columns[1:]:
        if col != compared_col:
            df[col] = df[col] - df[compared_col] 
    df[[compared_col]] = 0.00
    return df


def visualize_df(df, title):
    print(f"\n{title}\n")
    styled_df = df.style.set_table_styles(
    ).set_properties(**{
        'background-color': 'LightGray',
        'color': 'black',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'text-align': 'left'
    })
    

    # print(df.to_latex(escape=True))
    display(styled_df)
   

In [7]:


unique_datasets = pd.read_csv(os.path.join(os.getcwd(), 'results', 'log_likelihood_summary.csv'))['dataset'].unique()
dataset_info = pd.read_csv(os.path.join(repo_root, 'datasets', 'dataset_info.csv'))
filtered_dataset_info = dataset_info[dataset_info['Dataset_ID'].isin(unique_datasets)].set_index('Dataset_ID')

# print(dataset_info)
# domain_order = ["Election", "Sport", "Preferences", "Other"]
# filtered_dataset_info['Domain'] = pd.Categorical(filtered_dataset_info['Domain'], categories=domain_order, ordered=True)

filtered_dataset_info = filtered_dataset_info.sort_values(by=['K1', 'K2', 'R', 'Name'])
visualize_df(filtered_dataset_info, 'Datasets Tested')


Datasets Tested



,Name,N,M,R,K1,K2,K_avg
Dataset_ID,,,,,,,
6,APA_Election_2009,5,17469,3493.800000,1,5,3.764000
9,wolf,15,9671,644.733000,2,2,2.000000
2,UCL,173,651,3.763000,2,4,2.725000
1,FIFA_World_Cup,83,364,4.386000,2,4,2.659000
8,Letor,7,20,2.857000,2,6,3.900000
5,AGH_Course_Selection_2004,7,153,21.857000,7,7,7.000000
4,Sushi_100,100,5000,50.000000,10,10,10.000000
3,Sushi_10,10,5000,500.000000,10,10,10.000000
10,spotify_daily,21,45,2.143000,21,21,21.000000


In [11]:
log_like, leadership_log = process_results()

def rename_df(df):
    merged_df = df.merge(dataset_info, left_on='dataset', right_on='Dataset_ID').drop(columns=['Dataset_ID','dataset'])
    # merged_df = merged_df.rename(columns={'Name': 'dataset_name'}).set_index('dataset_name')

    domain_order = ["Election", "Sport", "Preferences", "Other"]
    # merged_df['Domain'] = pd.Categorical(merged_df['Domain'], categories=domain_order, ordered=True)
    merged_df = merged_df.sort_values(by=['K1', 'K2', 'Name'])
    merged_df = merged_df[[ 'Name', 'HO_BT','HOL_BT', 'BIN', 'BINL']]

    return merged_df

In [13]:

log_like = rename_df(log_like)
visualize_df(log_like, 'log likelihood')


log likelihood



,Name,HO_BT,HOL_BT,BIN,BINL
5,APA_Election_2009,0.000000,-0.033284,-0.000621,-0.024484
7,wolf,0.000000,0.000000,0.000000,-0.000000
0,FIFA_World_Cup,0.000000,-0.016749,-0.009852,-0.035207
1,UCL,0.000000,-0.039661,-0.006903,-0.056637
6,Letor,0.000000,-1.235648,-0.012993,-1.656084
4,AGH_Course_Selection_2004,0.000000,-0.698932,-0.014394,-0.698704
2,Sushi_10,0.000000,-1.404110,-0.051952,-1.419244
3,Sushi_100,0.000000,-1.061550,-0.033920,-1.322468
8,spotify_daily,0.000000,-7.497716,-0.704566,-12.538616


In [14]:
leadership_log = rename_df(leadership_log)
visualize_df(leadership_log, 'Leadership log likelihood')


Leadership log likelihood



,Name,HO_BT,HOL_BT,BIN,BINL
5,APA_Election_2009,-0.009030,0.000000,-0.007315,-0.000265
7,wolf,0.000000,0.000000,0.000000,-0.000000
0,FIFA_World_Cup,0.014407,0.000000,0.012610,-0.004050
1,UCL,0.015348,0.000000,0.015803,-0.005451
6,Letor,-0.439412,0.000000,-0.341989,0.025936
4,AGH_Course_Selection_2004,0.002735,0.000000,0.002801,0.000102
2,Sushi_10,-0.130627,0.000000,-0.085804,-0.000004
3,Sushi_100,-0.097688,0.000000,-0.063785,-0.004827
8,spotify_daily,-0.535823,0.000000,-0.319236,0.103068


In [15]:
log_likelihood_df = (
    pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'log_likelihood_summary.csv'))
    .groupby(by=['dataset'])
    .mean()
    .reset_index()
    .drop(columns=['epoch'])
)

log_likelihood_df.style.highlight_max(axis=1, color='grey', subset=log_likelihood_df.columns.difference(['dataset']))

,dataset,HO_BT,HOL_BT,BIN,BINL
0,1,-1.394630,-1.411379,-1.404482,-1.429837
1,2,-1.361104,-1.400765,-1.368006,-1.417741
2,3,-14.242772,-15.646882,-14.294724,-15.662016
3,4,-14.220160,-15.281710,-14.254080,-15.542628
4,5,-5.892606,-6.591538,-5.907000,-6.591309
5,6,-3.116870,-3.150153,-3.117491,-3.141353
6,8,-2.888960,-4.124607,-2.901952,-4.545044
7,9,-0.049859,-0.049859,-0.049859,-0.049859
8,10,-37.115602,-44.613318,-37.820168,-49.654218


In [16]:
log_likelihood_df = (
    pd.read_csv(os.path.join(EXPERIMENT_PATH, 'results', 'leadership_log_likelihood_summary.csv'))
    .groupby(by=['dataset'])
    .mean()
    .reset_index()
    .drop(columns=['epoch'])
)

log_likelihood_df.style.highlight_max(axis=1, color='grey', subset=log_likelihood_df.columns.difference(['dataset']))

,dataset,HO_BT,HOL_BT,BIN,BINL
0,1,-0.823295,-0.837703,-0.825092,-0.841753
1,2,-0.783808,-0.799155,-0.783352,-0.804607
2,3,-2.080218,-1.949592,-2.035396,-1.949596
3,4,-2.051544,-1.953856,-2.017641,-1.958682
4,5,-0.008217,-0.010953,-0.008152,-0.010850
5,6,-1.186001,-1.176971,-1.184286,-1.177236
6,8,-1.007507,-0.568095,-0.910084,-0.542159
7,9,-0.049859,-0.049859,-0.049859,-0.049859
8,10,-2.277984,-1.742161,-2.061397,-1.639093
